

---


Experiment with different variants of GANs using spectral normalization


---



In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch.nn as nn
import torch.nn.functional as F
from fastai.vision import *
from fastai.vision.gan import *

You should set the following option to True if the notebook isn't located in the file system inside a clone of the git repo (with the needed Python modules available) it belongs to; i.e., it's running independently.

In [ ]:
run_as_standalone_nb = False

In [ ]:
# This cell needs to be executed before importing local project modules, like import core.gan
if run_as_standalone_nb:
    root_lib_path = os.path.abspath('generative-lab')
    if not os.path.exists(root_lib_path):
        !git clone https://github.com/davidleonfdez/generative-lab.git
    if root_lib_path not in sys.path:
        sys.path.insert(0, root_lib_path)
else:
    import local_lib_import

In [ ]:
# Local project modules. Must be imported after local_lib_import or cloning git repo.
from core.gan import CustomGANLearner, GANLossArgs, gan_loss_from_func_std, load_gan_learner, save_gan_learner
from core.losses import hinge_adversarial_losses, hinge_like_adversarial_losses
from core.nb_utils import mount_gdrive
from core.net_builders import custom_critic

`models_root` is used as the base path to save models. Next cell sets assumes the nb is being executed from Google Colab and you have a "ML" dir in Google Drive. Alternatively, you could set it manually to something like './' to save and load models to/from the current directory.

In [ ]:
# Optional, allows saving parameters in gdrive
root_gdrive = mount_gdrive()
# Point this variable to the path where you want to save your models
models_root = root_gdrive + 'ML/'

In [ ]:
img_size = 64
img_n_channels = 3
batch_size = 128
use_cuda = torch.cuda.is_available()

# DATA

In [ ]:
ds_url = "http://vis-www.cs.umass.edu/lfw/lfw"

In [ ]:
realImagesPath = untar_data(ds_url)
realImagesPath

In [ ]:
def get_data(path, bs, size):
    return (GANItemList.from_folder(path, noise_sz=100)
               .split_none()
               .label_from_func(noop)
               .transform(tfms=[[crop_pad(size=size, row_pct=(0,1), col_pct=(0,1))], []], size=size, tfm_y=True)
               .databunch(bs=bs)
               .normalize(stats = [torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])], do_x=False, do_y=True))

In [ ]:
data = get_data(realImagesPath, batch_size, img_size)
data.show_batch()

# CRITIC

```
custom_critic(in_size:int, n_channels:int, n_features:int=64, 
              n_extra_layers:int=0, self_attention:bool=False, 
              **conv_kwargs) -> nn.Module
```

In [ ]:
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)

# GENERATOR

```
basic_generator(in_size:int, n_channels:int, noise_sz:int=100,  
                n_features:int=64, n_extra_layers=0, **conv_kwargs)
```

In [ ]:
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)

# TRAINING

* The parameters of a trained model can be saved with `save_gan_learner`.
* A training run can resumed (using weights saved during a previous session) with `load_gan_learner`. For example:
        load_gan_learner(learner, models_root + 'sngan-dcgan-Tr1a_60ep.pth')
    This must be executed after instantiating the learner and BEFORE running `learner.fit()`.

* Another alternative to launch a long training run is the method `save_checkpoint_gan`. It will automatically save the weights every `n_epochs_save_split` epochs.

## DCGAN (standard GAN with NSGAN loss function)

In [ ]:
def gen_loss_func(*args): return 0
crit_loss_func = nn.BCEWithLogitsLoss()

losses = gan_loss_from_func_std(gen_loss_func, crit_loss_func)

### TR 1: Spectral norm only in critic, batch norm in generator

#### TR 1a: lr=2e-4

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1a_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1a_60ep.pth')

#### TR 1b: lr=5e-4

In [ ]:
lr = 5e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1b_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1b_60ep.pth')

#### TR 1c: lr=2e-4, betas=(0.5,0.9)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.5,0.9)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1c_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1c_60ep.pth')

#### TR 1d: lr=2e-4, betas=(0.5,0.999)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.5,0.999)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1d_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1d_60ep.pth')

#### TR 1e: lr=2e-4, betas=(0.5,0.999), 1 disc/gen iterations

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func=partial(optim.Adam, betas = (0.5,0.999)), wd=0., 
                           switcher=partial(FixedGANSwitcher, n_crit=1, n_gen=1))

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1e_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1e_60ep.pth')

#### TR 1f: lr=2e-4, betas=(0.5,0.999), 2 disc/gen iterations

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func=partial(optim.Adam, betas = (0.5,0.999)), wd=0., 
                           switcher=partial(FixedGANSwitcher, n_crit=2, n_gen=1))

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1f_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1f_60ep.pth')

#### TR 1g: lr=2e-4, betas=(0,0.999)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                           opt_func=partial(optim.Adam, betas = (0.,0.999)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1g_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1g_60ep.pth')

#### TR 1h: lr=2e-4, hinge loss. Critic tries D(real)>=1, D(fake)<=-1. Gen tries D(fake)=inf.

**Hinge loss**

For real images, we want to minimize (real part of critic loss func): $-min(0, -1 + D(x))$
<br>$= max(0, 1-D(x))$

For fake images, we want to minimize (fake part of critic loss func): $-min(0, -1 - D(G(z)))$
<br>$= max(0, 1+D(G(z)))$

In [ ]:
hlosses = hinge_adversarial_losses()

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1h_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1h_60ep.pth')

#### TR 1i: lr=2e-4, pseudo hinge loss, only for fake preds. Critic tries D(real)=inf, D(fake)<=-1. Gen tries D(fake)=inf.

In [ ]:
hlosses = hinge_like_adversarial_losses(math.inf, math.inf, -1.) 

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1i_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1i_60ep.pth')

#### TR 1j: lr=2e-4, hinge loss with margin also in G loss. Critic tries D(real)>=1, D(fake)<=-1. Gen tries D(fake)>=1.

It shouldn't work, I believe the generator needs to push more, although it's the most symmetric formulation.

In [ ]:
hlosses = hinge_like_adversarial_losses(1., 1., -1.)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1j_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1j_60ep.pth')

#### TR 1k: lr=2e-4, hinge loss with irregular margin, real identification pushes more in critic. Critic tries D(real)>=10, D(fake)<=-1. Gen tries D(fake)=inf.

Maybe only the difference between the margins minds.

In [ ]:
hlosses = hinge_like_adversarial_losses(math.inf, 10., -1.)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1k_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1k_60ep.pth')

#### TR 1L: lr=2e-4, hinge loss with irregular margin, fake identification pushes more in critic. Critic tries D(real)>=1, D(fake)<=-10, gen tries D(fake)=inf

Maybe only the difference between the margins minds here.

In [ ]:
hlosses = hinge_like_adversarial_losses(math.inf, 1., -10.)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1l_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1l_60ep.pth')

#### TMP TR 1m: lr=2e-4, hinge loss with irregular margin, fake identification pushes more in critic. Critic tries D(real)>=1, D(fake)<=-10. Gen tries D(fake)>=1

In [ ]:
hlosses = hinge_like_adversarial_losses(1., 1., -10.) 

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1m_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1m_60ep.pth')

#### TMP TR 1n: lr=2e-4, hinge loss with irregular margin, fake identification pushes more in critic. Critic tries D(real)>=1, D(fake)<=-10. Gen tries D(fake) >= 12

In [ ]:
hlosses = hinge_like_adversarial_losses(12., 1., -10.)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1n_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1n_60ep.pth')

#### TR 1o: lr=2e-4, only hinge loss for real preds. Critic tries D(real)>=1, D(fake)=-inf. Gen tries D(fake)=inf

In [ ]:
hlosses = hinge_like_adversarial_losses(math.inf, 1., -math.inf)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1o_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1o_60ep.pth')

#### TR 1p: lr=2e-4, only hinge loss for real preds (1o with log(fake_pred) in critic loss). Critic tries D(real)>=1, D(fake)=-inf slow below 0 (minus sqrt). Gen tries D(fake)=inf

In [ ]:
[(x,-(abs(x)**(1/2))) for x in range(-10,10)]

In [ ]:
def hinge_adversarial_losses_p(margin:float=1.):
    def _loss_G(fake_pred, output, target):
        return -(fake_pred.mean())

    def _loss_C(real_pred, fake_pred):
        zero = torch.tensor([0.], device=real_pred.device)
        fake_pred_mean = fake_pred.mean()
        return (torch.max(zero, margin - real_pred).mean() + 
               torch.max(fake_pred_mean, -(torch.abs(fake_pred_mean)**(1/2))))

    return _loss_G, _loss_C

hlosses = hinge_adversarial_losses_p()

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1p_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1p_60ep.pth')

#### TR 1q: *(1j with crit/gen iters = 1/1)* lr=2e-4, hinge loss with margin also in G loss, 1 crit/gen iterations. Critic tries D(real)>=1, D(fake)<=-1. Gen tries D(fake)>=1.

In [ ]:
hlosses = hinge_like_adversarial_losses(1., 1., -1.)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.,
                           switcher=partial(FixedGANSwitcher, n_crit=1, n_gen=1))

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1q_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1q_60ep.pth')

#### TR 1r: (1h with crit/gen iters = 1/1) lr=2e-4, hinge loss. Critic tries D(real)>=1, D(fake)<=-1. Gen tries D(fake)=inf.

In [ ]:
hlosses = hinge_adversarial_losses()

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*hlosses), switch_eval=False, 
                           opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.,
                           switcher=partial(FixedGANSwitcher, n_crit=1, n_gen=1))

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1r_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr1r_60ep.pth')

### TR 2: Spectral norm in critic and generator

#### TR 2a: lr=2e-4

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                     opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr2a_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr2a_60ep.pth')

#### TR 2b: lr=5e-4

In [ ]:
lr = 5e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
learner = CustomGANLearner(data, generator, critic, GANLossArgs(*losses), switch_eval=False, 
                     opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr2b_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-dcgan-Tr2b_60ep.pth')

## WGAN

### TR 1: Spectral norm only in critic, batch norm in generator

#### TR 1a: lr=2e-4

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                          opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1a_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1a_60ep.pth')

#### TR 1b: lr=5e-4

In [ ]:
lr = 5e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1b_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1b_60ep.pth')

#### TR 1c: lr=2e-4, betas=(0.5,0.9)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.5,0.9)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1c_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1c_60ep.pth')

#### TR 1d: lr=2e-4, betas=(0.5,0.999)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.5,0.999)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1d_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1d_60ep.pth')

#### TR 1e: lr=2e-4, betas=(0,0.999)

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.999)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1e_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr1e_60ep.pth')

### TR 2: Spectral norm in critic and generator

#### TR 2a: lr=2e-4

In [ ]:
lr = 2e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                          opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr2a_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr2a_60ep.pth')

#### TR 2b: lr=5e-4

In [ ]:
lr = 5e-4
critic = custom_critic(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1, norm_type=NormType.Spectral)
learner = CustomGANLearner.wgan(data, generator, critic, clip=None, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr2b_30ep.pth')

In [ ]:
learner.fit(30, lr)
save_gan_learner(learner, models_root + 'sngan-wgan-Tr2b_60ep.pth')

# PENDING AND FINDINGS

Pending:
* Look into how to make spectral norm work in generator.

Facts:

* betas=(0.9,0.999), 1 disc/gen iterations doesn't work, at least with DCGAN. It's not common to see big beta1 values having a positive impact in GANs.
* WGAN (without GP) is difficult to tweak, and isn't even considered in the paper.

